# Data Engineer Challenge

### Instalación de liberías necesarias

In [1]:
# !pip install memory-profiler
# !pip install pandas
# !pip install line_profiler
# !pip install emoji

### Importación y setup

In [28]:
%load_ext memory_profiler
%load_ext line_profiler

import cProfile

from q1_memory import q1_memory
from q1_time import q1_time
from q2_memory import q2_memory
from q2_time import q2_time
from pandas import DataFrame as visualizador

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


### Definición de constantes

In [9]:
FILE_PATH = "../data/farmers-protest-tweets-2021-2-4.json"

---
### ✅ Supuestos realizados

- Se asumió que el campo de fecha está en `"date"` en formato ISO 8601.
- El nombre de usuario se encuentra en `"user.username"`.
- Registros con campos faltantes fueron ignorados.

# Pregunta 1 - Tweets por Fecha y Usuario más Activo
> **Objetivo:** Obtener las 10 fechas con más tweets y, para cada una de esas fechas, el usuario más activo (con más publicaciones).

## Enfoque de memoria optimizada

Para minimizar el uso de memoria, implementa las siguientes estrategias:

- Procesa el archivo en modo **streaming** (línea por línea), evitando cargar todo el contenido en memoria.
- Utiliza un **heap limitado** (`min-heap`) para mantener únicamente las **10 fechas con más tweets**, descartando automáticamente las menos relevantes.
- Evita almacenar conteos de todas las fechas posibles, lo que reduce significativamente el uso de memoria frente a otras alternativas.
- Solo conserva información de conteo de usuarios por fecha, necesaria para determinar el usuario más activo en cada una de las 10 fechas seleccionadas.
- Ignora registros incompletos (sin fecha o sin username).
- Convierte las fechas a tipo `datetime.date` solo al final, una vez determinadas las fechas relevantes.


#### Prueba de memoria

In [8]:
%memit q1_memory(FILE_PATH)

peak memory: 112.70 MiB, increment: 2.81 MiB


#### Prueba de tiempo

In [17]:
%timeit -n 10 -r 3 q1_memory(FILE_PATH)

3.47 s ± 40.1 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


#### Resultado

In [11]:
fecha_y_usuarios = q1_memory(FILE_PATH)
visualizador(fecha_y_usuarios, columns=["Fecha", "Usuarios"])

,Fecha,Usuarios
0,2021-02-12,RanbirS00614606
1,2021-02-13,MaanDee08215437
2,2021-02-17,RaaJVinderkaur
3,2021-02-16,jot__b
4,2021-02-14,rebelpacifist
5,2021-02-18,neetuanjle_nitu
6,2021-02-15,jot__b
7,2021-02-20,MangalJ23056160
8,2021-02-23,Surrypuria
9,2021-02-19,Preetm91


## Enfoque de tiempo optimizado

Para minimizar el tiempo de ejecución, esta versión de la función implementa las siguientes estrategias:

- Carga **todo el archivo** de tweets completamente en memoria, reduciendo el tiempo asociado a operaciones de lectura desde disco (I/O).
- Emplea estructuras eficientes (`Counter` y `defaultdict`) para realizar conteos y agrupaciones rápidamente en memoria.
- Evita validaciones o transformaciones costosas dentro del loop principal, procesando los datos de forma directa.
- Realiza las conversiones de strings a `datetime.date` **solamente al final**, una vez que ya se han identificado las 10 fechas con más tweets.
- Ignora registros incompletos (sin fecha o sin username) para evitar errores o procesamiento innecesario.


#### Prueba de memoria

In [27]:
%memit q1_time(FILE_PATH)

peak memory: 124.94 MiB, increment: 2.19 MiB


#### Prueba de tiempo

In [32]:
%timeit -n 10 -r 3 q1_time(FILE_PATH)

1.23 s ± 15.8 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


#### Resultado

In [28]:
fecha_y_usuarios = q1_time(FILE_PATH)
visualizador(fecha_y_usuarios, columns=["Fecha", "Usuarios"])

,Fecha,Usuarios
0,2021-02-12,RanbirS00614606
1,2021-02-13,MaanDee08215437
2,2021-02-17,RaaJVinderkaur
3,2021-02-16,jot__b
4,2021-02-14,rebelpacifist
5,2021-02-18,neetuanjle_nitu
6,2021-02-15,jot__b
7,2021-02-20,MangalJ23056160
8,2021-02-23,Surrypuria
9,2021-02-19,Preetm91


---
# Pregunta 2 - Los top 10 emojis más usados
> **Objetivo:** Obtener los 10 emojis más usados con su respectivo conteo

## Enfoque de memoria optimizada

Para minimizar el uso de memoria, esta versión de la función implementa las siguientes estrategias:

- Procesa el archivo de tweets en modo **streaming** (línea por línea), evitando cargar el archivo completo en memoria.
- Analiza únicamente el campo de texto (`content`) de cada tweet, sin almacenar los textos ni listas completas.
- Utiliza estructuras simples y eficientes (diccionario plano) para contar ocurrencias de emojis.
- Recorre el texto carácter por carácter para identificar emojis de forma directa, sin operaciones costosas adicionales.
- Solo mantiene en memoria el diccionario de conteo acumulado de emojis, que tiende a ser pequeño.
- Ordena y extrae los 10 emojis más frecuentes una vez finalizado el procesamiento completo.

#### Prueba de memoria

In [10]:
%memit q2_memory(FILE_PATH)

peak memory: 110.01 MiB, increment: 1.06 MiB


#### Prueba de tiempo

In [11]:
%timeit -n 10 -r 3 q2_memory(FILE_PATH)

5.74 s ± 114 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


#### Respuesta

In [16]:
emojis_y_conteo = q2_memory(FILE_PATH)
visualizador(emojis_y_conteo, columns=["Emojis", "Conteos"])

,Emojis,Conteos
0,🙏,7286
1,😂,3072
2,🚜,2972
3,✊,2411
4,🌾,2363
5,🏻,2080
6,❤,1779
7,🤣,1668
8,🏽,1218
9,👇,1108


## Enfoque de tiempo optimizado

Para minimizar el tiempo de ejecución, esta versión de la función implementa las siguientes estrategias:

- Carga **todo el archivo** en memoria utilizando `readlines()`, lo que reduce el tiempo de acceso al disco (I/O).
- Utiliza la estructura `set` construida a partir de `emoji.EMOJI_DATA.keys()` para identificar rápidamente si un carácter es un emoji, con operaciones de búsqueda en tiempo constante.
- Emplea `Counter` de la librería estándar para acumular los conteos de emojis de forma eficiente y rápida.
- Recorre el texto carácter por carácter, sin validaciones adicionales innecesarias dentro del bucle.
- Ignora líneas con errores de decodificación y tweets sin campo `content` para evitar procesamiento extra.
- Extrae los 10 emojis más frecuentes utilizando el método `most_common()` de `Counter`, que es altamente optimizado.


#### Prueba de memoria

In [50]:
%memit q2_time(FILE_PATH)

peak memory: 584.54 MiB, increment: 162.01 MiB


#### Prueba de tiempo

In [49]:
%timeit -n 10 -r 3 q2_time(FILE_PATH)

4.2 s ± 307 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


#### Respuesta

In [47]:
emojis_y_conteo = q2_time(FILE_PATH)
visualizador(emojis_y_conteo, columns=["Emojis", "Conteos"])

,Emojis,Conteos
0,🙏,7286
1,😂,3072
2,🚜,2972
3,✊,2411
4,🌾,2363
5,🏻,2080
6,❤,1779
7,🤣,1668
8,🏽,1218
9,👇,1108
